In [1]:
# graph_reasoning_colab.ipynb (Python code cells below are meant for notebook)

# Step 1: Install Required Packages
!pip install networkx huggingface_hub pandas numpy scikit-learn openai

# Step 2: Import Libraries
import networkx as nx
import pandas as pd
import numpy as np
import pickle
from huggingface_hub import hf_hub_download
from sklearn.metrics.pairwise import cosine_similarity
import openai

# Step 3: Load the Graph
repo_id = 'lamm-mit/bio-graph-1K'
graph_file = hf_hub_download(repo_id=repo_id, filename='large_graph_simple_giant.graphml')
G = nx.read_graphml(graph_file)
print(f"Graph loaded with {len(G.nodes())} nodes and {len(G.edges())} edges.")

# Step 4: Load the Embeddings
embedding_file = hf_hub_download(repo_id=repo_id, filename='embeddings_simple_giant_ge-large-en-v1.5.pkl')
with open(embedding_file, 'rb') as f:
    embeddings = pickle.load(f)

# Step 5: Node Similarity Function
def get_similar_nodes(node_id, top_k=5):
    if node_id not in embeddings:
        return []
    vec = embeddings[node_id].reshape(1, -1)
    keys = list(embeddings.keys())
    mat = np.stack([embeddings[k] for k in keys])
    sims = cosine_similarity(vec, mat)[0]
    top_indices = np.argsort(-sims)[1:top_k+1]
    return [(keys[i], sims[i]) for i in top_indices]

# Step 6: Shortest Path Finder
def find_path(start_node, end_node):
    if start_node not in G or end_node not in G:
        return "Invalid node(s)"
    try:
        path = nx.shortest_path(G, source=start_node, target=end_node)
        return path
    except nx.NetworkXNoPath:
        return "No path found"

# Step 7: GPT-Based Reasoning (Optional)
def query_gpt(prompt, model="gpt-4", temperature=0.7):
    openai.api_key = "sk-..."  # Replace with your API key
    response = openai.ChatCompletion.create(
        model=model,
        messages=[{"role": "system", "content": "You are a materials science reasoning assistant."},
                 {"role": "user", "content": prompt}],
        temperature=temperature
    )
    return response.choices[0].message['content']

# Example Use
# path = find_path("mineralized collagen fibers", "schreger pattern")
# print(path)

# similar_nodes = get_similar_nodes("hydroxyapatite")
# print(similar_nodes)

# response = query_gpt("What is the role of collagen in hierarchical biological structures?")
# print(response)


  Attempting uninstall: packaging
    Found existing installation: packaging 20.4
    Uninstalling packaging-20.4:
      Successfully uninstalled packaging-20.4


c:\Users\rgutkin\Anaconda3\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rgutkin\.cache\huggingface\hub\models--lamm-mit--bio-graph-1K. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Graph loaded with 33159 nodes and 48753 edges.
